In [1]:
import os 
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)


True

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='./temec_db',
                       collection_name='temec', collection_metadata={"temec:space": "temec"})        

In [3]:
db.get(include=['embeddings', 'metadatas', 'documents'])


{'ids': ['d842385d-bc36-4f82-a078-768e82e78937',
  '155d5c09-6b4a-414d-982f-03ede2997e6a',
  '8365c1b5-933c-4079-857a-01d38c842f49',
  '95decabd-839d-4144-a025-a8f42171e4f0',
  'ea45a6f5-08c0-4ac3-a810-111ae7d8fdd3',
  '19d17327-8d24-45cd-96d2-7ac57b773e54',
  '84644c2b-e5f9-4474-acc3-ae6db63d9c15',
  'f90b648d-caed-4482-980f-002d9dffab5b',
  '170ba557-9362-4524-bcc2-1de1a2e9056a',
  'cde3d1a4-19b8-47a5-82f1-4ceef19d09ab',
  'b6e7b74d-0579-4f3e-b5bf-cf548f16328d',
  'b4d1cc00-4a59-4a98-b14d-68ca82842ee3',
  'e9dbcbe5-d3b7-4167-a05b-150908106446',
  '56e480ec-b600-4355-981b-b9bf028af8b1',
  '775704ab-5507-4113-8d72-4d7f81a663a0',
  'ca88e0e9-f58c-4661-89fa-b0657036cb22',
  'c34098f8-7901-47d6-ad05-85288a90d483',
  '7f455c61-d70f-4b1e-8abe-645b39e1b850',
  '23943a86-9d50-4feb-8f37-9ac5ab8f9662',
  '141b953f-db6b-4ce2-8662-522924959815',
  '1298ceca-929c-4713-9b44-9d7df86fb5b6',
  '873a2f1e-fc43-4e2f-a076-9ad8ac3605c3',
  'bd6a4bc7-8cee-4ca4-beb4-165abb5a86a0',
  'fcf0bed2-320e-4deb-ac19-

In [4]:
retriever=db.as_retriever(search_kwargs={"k":2})
retriever_results=retriever.invoke("Cuanto debe de pagar el 1 tonelada de maiz")
print(retriever_results)

[Document(id='0fa1c241-553e-4b47-a0b8-7ff77b973bd8', metadata={'page': 35, 'source': 'docs/T-MEC.pdf'}, page_content='4. El arancel aduanero aplicado por Estados Unidos a las autopartes importadas desde México \nlistadas en el Apéndice 1 al Anexo 4 -B (Lista de Fracciones Arancelarias) que no califican como \noriginarias conforme a las reglas de origen del Capítulo 4 de este Acuerdo, no excederá el menor \nde, la tasa del aran cel NMF aplicada por Estados Unidos en vigor al 1° de agosto de  2018 o del \narancel NMF aplicado por Estados Unidos al momento de la importación de la mercancía.'), Document(id='498f8060-9a49-416d-97dd-4c288c3351a0', metadata={'page': 35, 'source': 'docs/T-MEC.pdf'}, page_content='4. El arancel aduanero aplicado por Estados Unidos a las autopartes importadas desde México \nlistadas en el Apéndice 1 al Anexo 4 -B (Lista de Fracciones Arancelarias) que no califican como \noriginarias conforme a las reglas de origen del Capítulo 4 de este Acuerdo, no excederá el m

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0.7)

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ✅ Adaptación del template al contexto aduanero
template = """
Responde la siguiente pregunta con base únicamente en el siguiente contexto relacionado con procesos aduaneros:

{context}

Pregunta: {question}
Respuesta:
"""

prompt = ChatPromptTemplate.from_template(template)

# ✅ Conversión de documentos a texto simple
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# ✅ Cadena RAG
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [18]:
question = "¿Cuáles son los requisitos para importar honda 2003 usada desde Estados Unidos?"
respuesta = rag_chain.invoke(question)
print(respuesta)


Para importar un vehículo Honda 2003 usado desde Estados Unidos, según el contexto proporcionado, se deben cumplir con las medidas de seguridad y de emisiones para vehículos automotores y los requisitos de registro vehicular que sean de aplicación general en el país importador. Estas medidas deben ser consistentes con el acuerdo mencionado, pero no se detallan requisitos específicos más allá de esto en el contexto proporcionado.


In [19]:
question = "¿Que autos no puedo importar desde Estados Unidos?"
respuesta = rag_chain.invoke(question)
print(respuesta)


Con base únicamente en el contexto proporcionado, no se especifica de manera clara qué autos no se pueden importar desde Estados Unidos. El texto hace referencia a ciertas disposiciones del Acuerdo de Comercio Exterior de México, específicamente los párrafos 1(I) y 5 del Anexo 2.2.1, pero no detalla los tipos de vehículos sujetos a restricciones. Para conocer exactamente qué vehículos no se pueden importar, sería necesario consultar el texto completo del Acuerdo mencionado o las regulaciones detalladas en esos párrafos específicos.


# 🔁 Adaptación del flujo Agentic RAG a temas de Aduanas
🔧 1. Preprocesar documentos aduanales
Puedes usar documentos como:

Manual del Agente Aduanal

Ley Aduanera (SAT, ANAM)

Procedimientos de importación/exportación

Documentos PDF de pedimentos, Incoterms, etc.

In [21]:
from langchain_community.document_loaders import PyPDFLoader

docs = [
    PyPDFLoader("docs/manual_agente_aduanal.pdf").load(),
    PyPDFLoader("docs/ley_aduanera_mexico.pdf").load()
]

# Flatten
docs = [item for sublist in docs for item in sublist]


In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs)


In [24]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_aduana_docs",
    "Busca y devuelve información relevante sobre legislación y procedimientos aduanales."
)


In [26]:
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState

response_model = ChatOpenAI()

def generate_query_or_respond(state: MessagesState):
    return {
        "messages": [
            response_model.bind_tools([retriever_tool]).invoke(state["messages"])
        ]
    }


In [28]:
from pydantic import BaseModel, Field
from typing import Literal

class GradeDocuments(BaseModel):
    binary_score: str = Field(description="'yes' si es relevante, 'no' si no lo es")

GRADE_PROMPT = """
Evalúa si el documento es relevante para la pregunta del usuario sobre temas aduanales.

Documento: {context}
Pregunta: {question}
¿Es relevante? ('yes' o 'no')
"""

grader_model = ChatOpenAI()

def grade_documents(state: MessagesState) -> Literal["generate_answer", "rewrite_question"]:
    question = state["messages"][0].content
    context = state["messages"][-1].content

    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = grader_model.with_structured_output(GradeDocuments).invoke(
        [{"role": "user", "content": prompt}]
    )
    return "generate_answer" if response.binary_score == "yes" else "rewrite_question"


In [29]:
REWRITE_PROMPT = """
Aquí está la pregunta original:
{question}
Por favor reformúlala para que sea más clara y se entienda mejor el contexto aduanal.
"""

def rewrite_question(state: MessagesState):
    question = state["messages"][0].content
    prompt = REWRITE_PROMPT.format(question=question)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [{"role": "user", "content": response.content}]}


In [31]:
GENERATE_PROMPT = """
Eres un asistente experto en comercio exterior. Responde con base en el siguiente contexto aduanal:

Pregunta: {question}
Contexto: {context}

Responde en máximo tres frases.
"""

def generate_answer(state: MessagesState):
    """Genera una respuesta final basada en la pregunta y el contexto."""
    question = state["messages"][0].content
    context = state["messages"][-1].content
    prompt = GENERATE_PROMPT.format(question=question, context=context)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [response]}


In [32]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

workflow = StateGraph(MessagesState)
workflow.add_node(generate_query_or_respond)
workflow.add_node("retrieve", ToolNode([retriever_tool]))
workflow.add_node(rewrite_question)
workflow.add_node(generate_answer)

workflow.add_edge(START, "generate_query_or_respond")

workflow.add_conditional_edges(
    "generate_query_or_respond",
    tools_condition,
    {"tools": "retrieve", END: END},
)

workflow.add_conditional_edges("retrieve", grade_documents)
workflow.add_edge("generate_answer", END)
workflow.add_edge("rewrite_question", "generate_query_or_respond")

graph = workflow.compile()


In [33]:
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "¿Qué documentos necesito para papaiptas fir?"}]}
):
    for node, update in chunk.items():
        print(f"🧩 Nodo: {node}")
        update["messages"][-1].pretty_print()
        print("\n")


🧩 Nodo: generate_query_or_respond
================================== Ai Message ==================================
Tool Calls:
  retrieve_aduana_docs (call_lM6o7ts7CSL3icx8uNSmIrrf)
 Call ID: call_lM6o7ts7CSL3icx8uNSmIrrf
  Args:
    query: documentos para importar una máquina industrial




/home/user/langchain/.venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1686: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


🧩 Nodo: retrieve
================================= Tool Message =================================
Name: retrieve_aduana_docs

sistemas de control de inventarios ligados a su contabilidad. 
Al presentarse los vehículos que serán motivo de exportación ante la aduana de salida, se deberá presentar la factura o la re lación de 
facturas que correspondan a la exportación. 
Adicionalmente las empresas podrán entregar a su s proveedores maquinaria y equipo especial sujeto a depósito fiscal para realizar 
procesos de fabricación de piezas automotrices, siempre que dichos proveedores se encuentren incluidos en los listados o

pedimento. 
Tratándose de maquinaria y equipo especial, dicho reconocimiento podrá realizars e por la aduana de la jurisdicción en el domicilio del 
importador, previa solicitud y autorización por parte de la Aduana.  
4.3.2.5 Las empresas de la industria automotriz terminal o manufacturera de vehículos de autotransporte que realicen la introducción al 
territorio nacional

AttributeError: 'dict' object has no attribute 'pretty_print'